In [1]:
import json
import glob
import os
import calendar

In [2]:
import numpy as np 
import pandas as pd                  # for DataFrames
from numpy.linalg import eig
import matplotlib.pyplot as plt

In [3]:
from sklearn import preprocessing
import scipy.fftpack                 # discrete Fourier transforms
from scipy import signal
from scipy.signal import find_peaks
from scipy.signal import butter, lfilter
from scipy.signal import find_peaks, peak_prominences
from scipy.signal import chirp, peak_widths

In [4]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots


In [5]:
import re

In [9]:
# Directory path
day = "Emulator/05-02-2024"
directory_path = "/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/Project_Carehub/Carewear-DataTesting2024/" + day


In [10]:
def file_count(directory_path, specific_string):
    # Get all files in the directory
    files = os.listdir(directory_path)
    # Filter files containing the specific string in the name
    filtered_files = [file for file in files if specific_string in file]
    # Initialize an empty Pandas DataFrame
    df = pd.DataFrame()
    # Loop through the filtered files and read them into the DataFrame
    for file in filtered_files:
        file_path = os.path.join(directory_path, file)
        # Assuming the files are CSV, you can adjust the read function accordingly
        temp_df = pd.read_csv(file_path)
        df = pd.concat([df, temp_df], ignore_index=True)
    # Display the DataFrame
    print(df)
    # Display the count of files containing the specific string
    print(f"Number of files containing '{specific_string}': {len(filtered_files)}")
    return len(filtered_files)

In [11]:
file_count_acc = file_count(directory_path, "acc")

          X         Y         Z event.timestamp                Timestamp
0       0.0  9.776321  0.812345                  2024-02-05 14:05:11.660
1       0.0  9.776321  0.812345                  2024-02-05 14:05:11.693
2       0.0  9.776321  0.812345                  2024-02-05 14:05:11.726
3       0.0  9.776321  0.812345                  2024-02-05 14:05:11.760
4       0.0  9.776321  0.812345                  2024-02-05 14:05:11.791
...     ...       ...       ...             ...                      ...
108643  0.0  9.776321  0.812345                  2024-02-05 14:20:11.985
108644  0.0  9.776321  0.812345                  2024-02-05 14:20:12.017
108645  0.0  9.776321  0.812345                  2024-02-05 14:20:12.054
108646  0.0  9.776321  0.812345                  2024-02-05 14:20:12.084
108647  0.0  9.776321  0.812345                  2024-02-05 14:20:12.116

[108648 rows x 5 columns]
Number of files containing 'acc': 13


In [13]:
file_count_gry = file_count(directory_path, "gry")

          X    Y    Z  event.timestamp                Timestamp
0       0.0  0.0  0.0   18358960109512  2024-02-05 13:30:10.207
1       0.0  0.0  0.0   18358993442512  2024-02-05 13:30:10.243
2       0.0  0.0  0.0   18359026775512  2024-02-05 13:30:10.276
3       0.0  0.0  0.0   18359060108512  2024-02-05 13:30:10.309
4       0.0  0.0  0.0   18359093441512  2024-02-05 13:30:10.342
...     ...  ...  ...              ...                      ...
108015  0.0  0.0  0.0   18959054108512  2024-02-05 13:40:10.302
108016  0.0  0.0  0.0   18959087441512  2024-02-05 13:40:10.337
108017  0.0  0.0  0.0   18959120774512  2024-02-05 13:40:10.370
108018  0.0  0.0  0.0   18959154107512  2024-02-05 13:40:10.403
108019  0.0  0.0  0.0   18959187440512  2024-02-05 13:40:10.435

[108020 rows x 5 columns]
Number of files containing 'gry': 13


In [14]:
file_count_hr = file_count(directory_path, "heart_rate")

    HeartRate                Timestamp
0         0.0  2024-02-05 13:25:09.653
1         0.0  2024-02-05 13:25:09.653
2         0.0  2024-02-05 13:25:09.653
3         0.0  2024-02-05 13:25:09.653
4         0.0  2024-02-05 13:25:09.653
5         0.0  2024-02-05 13:25:09.653
6         0.0  2024-02-05 13:25:09.653
7         0.0  2024-02-05 13:25:09.653
8         0.0  2024-02-05 13:25:09.653
9         0.0  2024-02-05 13:25:09.653
10        0.0  2024-02-05 13:25:09.653
11        0.0  2024-02-05 13:25:09.653
12        0.0  2024-02-05 13:25:09.653
Number of files containing 'heart_rate': 13
